-> Kernel Crashed also RAM crashed on colab that's why outputs are lost.  886m running
1st crashed during HOG feature extraction
2nd crashed during SVM Training
3rd crashed during evaluation

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Dataset:
the dataest have 10 class of fruits :

apple
mango
banana
cherry
pinenapple
strawberries
watermelon
avocado
kiwi
orange

230 image per fruit for training
110 image per fruit for test

Data Preparation:


Essential libraries such as os, cv2, numpy, sklearn, and PIL are imported for file operations, image processing, numerical computations, machine learning utilities, and image loading respectively.

load_images: Loads images from the specified folder, resizes them, converts to RGB format, and returns them along with their labels.

In [2]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from PIL import Image

def load_images(folder, target_size=(224, 224)):  # Set a target size, for example, (224, 224)
    images = []
    labels = []
    for fruit_class, fruit_name in enumerate(sorted(os.listdir(folder))):
        fruit_dir = os.path.join(folder, fruit_name)
        if os.path.isdir(fruit_dir):  # Check if it's a directory
            for filename in os.listdir(fruit_dir):
                img_path = os.path.join(fruit_dir, filename)
                try:
                    img = Image.open(img_path)
                    img = img.convert("RGB")  # Convert to RGB format
                    img = img.resize(target_size)  # Resize the image
                    img = np.array(img)  # Convert to numpy array
                    images.append(img)
                    labels.append(fruit_class)
                except Exception as e:
                    print(f"Failed to load image: {img_path} - Error: {e}")
    return np.array(images), np.array(labels)


# Function to prepare data
def prepare_data(train_folder, test_folder):
    # Load training images and labels
    train_images, train_labels = load_images(train_folder)
    # Load test images and labels
    test_images, test_labels = load_images(test_folder)
    # Shuffle training data
    train_images, train_labels = shuffle(train_images, train_labels, random_state=42)
    return train_images, train_labels, test_images, test_labels

# Set paths to train and test folders
train_folder = "/content/drive/MyDrive/Dataset/train"
test_folder = "/content/drive/MyDrive/Dataset/test"

# Prepare data
train_images, train_labels, test_images, test_labels = prepare_data(train_folder, test_folder)



In [ ]:
%pip install opencv-python numpy sklearn.utils

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 466.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/40.4 kB ? eta -:--:--
     ------------------------------ --------- 30.7/40.4 kB 1.3 MB/s eta 0:00:01
     ------------------------------ --------- 30.7/40.4 kB 1.3 MB/s eta 0:00:01
     ---------------

Image Augmentation:

augment_images: Augments images by applying transformations (like flipping) to increase the size of the training dataset.

In [3]:
# Function for image augmentation
def augment_images(images, labels, augmentations=5):
    augmented_images = []
    augmented_labels = []
    for image, label in zip(images, labels):
        for _ in range(augmentations):
            augmented_img = image.copy()
            # Apply random augmentation such as rotation, flipping, and scaling
            # Example:
            # augmented_img = cv2.rotate(augmented_img, cv2.ROTATE_90_CLOCKWISE)
            augmented_img = cv2.flip(augmented_img, 1)
            # augmented_img = cv2.resize(augmented_img, (new_width, new_height))
            augmented_images.append(augmented_img)
            augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

# Augment training images
augmented_train_images, augmented_train_labels = augment_images(train_images, train_labels)


Image Preprocessing:

Normalizes images by scaling pixel values.

In [4]:
# Function for image preprocessing
def preprocess_images(images):
    preprocessed_images = []
    for img in images:
        # Apply preprocessing techniques such as normalization, histogram equalization, and denoising
        # Example:
        #normalized_img = img / 255.0
        # equalized_img = cv2.equalizeHist(img)
        denoised_img = cv2.GaussianBlur(img, (5, 5), 0)
        preprocessed_images.append(denoised_img)
    return np.array(preprocessed_images)

# Preprocess augmented training images
preprocessed_train_images = preprocess_images(augmented_train_images)
# Preprocess test images
preprocessed_test_images = preprocess_images(test_images)


Feature Extraction by using HOG:

Functions to resize images and extract Histogram of Oriented Gradients (HOG) features using multiprocessing for efficiency.

In [ ]:
# Function for feature extraction
import cv2
import numpy as np
from multiprocessing import Pool

def resize_image(image, size=(64, 128)):
    return cv2.resize(image, size)

def process_image(image):
    # Resize image to a smaller size to reduce computation
    image = resize_image(image)

    # Convert image to the correct format (if needed)
    if image.dtype != 'uint8':
        image = cv2.convertScaleAbs(image)

    # Ensure the image is in grayscale
    if image.ndim == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create HOG descriptor with parameters for faster computation
    hog = cv2.HOGDescriptor(_winSize=(64, 128), _blockSize=(16, 16), _blockStride=(8, 8), _cellSize=(8, 8), _nbins=9)

    # Compute HOG features
    features = hog.compute(image)
    return features.flatten()

def compute_hog_features(images):
    # Use multiprocessing to compute HOG features in parallel
    with Pool() as pool:
        hog_features = pool.map(process_image, images)
    return hog_features

def extract_features(images):
    hog_features = compute_hog_features(images)
    return hog_features

# Example usage:
# Assuming preprocessed_train_images and preprocessed_test_images are defined
train_features = extract_features(preprocessed_train_images)
test_features = extract_features(preprocessed_test_images)



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Feature Extraction by using CNN:

Functions to preprocess images for CNN input, use a pre-trained VGG16 model to extract features, and handle image resizing and color space conversions.

In [5]:
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from multiprocessing import Pool

def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size)

def preprocess_image(image):
    # Resize image to the size expected by the CNN model
    image = resize_image(image)

    # Ensure the image is in RGB format
    if image.ndim == 2:  # If the image is grayscale
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.ndim == 3 and image.shape[2] == 1:  # If the image has a single channel
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.ndim == 3 and image.shape[2] == 4:  # If the image has an alpha channel
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)

    # Convert image to float32 and preprocess for VGG16
    image = image.astype('float32')
    image = preprocess_input(image)

    return image

def extract_cnn_features(model, images):
    # Use multiprocessing to preprocess images in parallel
    with Pool() as pool:
        preprocessed_images = pool.map(preprocess_image, images)

    # Convert list to numpy array
    preprocessed_images = np.array(preprocessed_images)

    # Extract features using the CNN model
    features = model.predict(preprocessed_images)

    return features

def create_feature_extractor():
    # Load the VGG16 model, pre-trained on ImageNet
    base_model = VGG16(weights='imagenet', include_top=False)
    # Create a new model that outputs features from the last convolutional layer
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    return model

# Create the feature extraction model
cnn_model = create_feature_extractor()

def extract_features(images):
    cnn_features = extract_cnn_features(cnn_model, images)
    return cnn_features

# Example usage:
# Assuming preprocessed_train_images and preprocessed_test_images are defined
train_features = extract_features(preprocessed_train_images)
test_features = extract_features(preprocessed_test_images)


58889256/58889256 [==============================] - 4s 0us/step


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


10/10 [==============================] - 4s 449ms/step


Dimensionality Reduction with PCA:

reduce_dimensionality: Reduces feature dimensionality using Principal Component Analysis (PCA).

In [6]:
import numpy as np
from sklearn.decomposition import PCA

# Function for dimensionality reduction using PCA
def reduce_dimensionality(features, n_components=50):
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    return reduced_features

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

# Reduce dimensionality of extracted features using PCA
n_components = 50  # Adjust the number of components as needed
reduced_train_features = reduce_dimensionality(train_features_flat, n_components)
reduced_test_features = reduce_dimensionality(test_features_flat, n_components)

# Check the shapes of the reduced features
print(f"Reduced train features shape: {reduced_train_features.shape}")
print(f"Reduced test features shape: {reduced_test_features.shape}")


Reduced train features shape: (3450, 50)
Reduced test features shape: (300, 50)


Dimensionality Reduction with LDA:

reduce_dimensionality_lda: Reduces feature dimensionality using Linear Discriminant Analysis (LDA), fitting the model on training data and transforming test data.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Function for dimensionality reduction using LDA
def reduce_dimensionality_lda(train_features, train_labels, n_components=50):
    lda = LDA(n_components=n_components)
    reduced_train_features = lda.fit_transform(train_features, train_labels)
    return lda, reduced_train_features

# Reduce dimensionality of extracted features using LDA
n_components = 50  # Adjust the number of components as needed

# Fit and transform the training features
lda, reduced_train_features = reduce_dimensionality_lda(train_features, train_labels, n_components)

# Transform the test features using the fitted LDA
reduced_test_features = lda.transform(test_features)


ValueError: Found array with dim 4. LinearDiscriminantAnalysis expected <= 2.

Dimensionality Reduction and predictions with QDA:

fit_qda: Fits a Quadratic Discriminant Analysis (QDA) model on training data and makes predictions on both training and test data.

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

# Function to fit QDA model and predict
def fit_qda(train_features, train_labels):
    qda = QDA()
    qda.fit(train_features, train_labels)
    return qda

# Ensure that train_features and train_labels have consistent lengths
print(f"Shape of train_features: {train_features.shape}")
print(f"Length of train_labels: {len(train_labels)}")

# Example code to ensure the number of features and labels match
if train_features.shape[0] != len(train_labels):
    raise ValueError(f"Number of samples in train_features ({train_features.shape[0]}) and train_labels ({len(train_labels)}) do not match!")

# Fit QDA model
qda_model = fit_qda(train_features, train_labels)

# Predict on train and test sets
train_predictions = qda_model.predict(train_features)
test_predictions = qda_model.predict(test_features)

# Print predictions
print("Train Predictions:", train_predictions)
print("Test Predictions:", test_predictions)



Shape of train_features: (3450, 7, 7, 512)
Length of train_labels: 690


ValueError: Number of samples in train_features (3450) and train_labels (690) do not match!

Training SVM Classifier:

train_classifier: Trains a Support Vector Machine (SVM) classifier on the reduced feature set.

In [7]:
from sklearn.svm import SVC

# Function for training SVM classifier
def train_classifier(features, labels):
    svm = SVC(kernel='linear')
    svm.fit(features, labels)
    return svm

# Train SVM classifier using reduced features and augmented labels
classifier = train_classifier(reduced_train_features, augmented_train_labels)


Evaluation:

evaluate_classifier: Evaluates the classifier's performance on the test set, reporting accuracy and a detailed classification report.

In [8]:
from sklearn.metrics import classification_report, accuracy_score

# Function for evaluation
def evaluate_classifier(classifier, test_features, test_labels):
    predictions = classifier.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    report = classification_report(test_labels, predictions)
    return accuracy, report

# Evaluate classifier using reduced test features and test labels
accuracy, report = evaluate_classifier(classifier, reduced_test_features, test_labels)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


Accuracy: 0.36666666666666664
Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.54      0.47        89
           1       0.34      0.31      0.33       105
           2       0.33      0.27      0.30       106

    accuracy                           0.37       300
   macro avg       0.36      0.38      0.37       300
weighted avg       0.36      0.37      0.36       300

